# debug issue with interpolation with parcels simulations

See [GH issue](https://github.com/OceanParcels/parcels/issues/1122)

In [1]:
import numpy as np
import xarray as xr
from parcels import FieldSet, ParticleSet, JITParticle, Variable, AdvectionRK4, ParticleFile

INFO: Compiled ParcelsRandom ==> /var/folders/pb/vp119fcj1pn3dc3x2dz4dkt80000gn/T/parcels-501/libparcels_random_bfa75db7-25e5-46d7-afd4-6a657983c50f.so


In [2]:
class SampleParticle(JITParticle):
    p = Variable('p', dtype=np.float32)

def SampleP(particle, fieldset, time):
    particle.p = fieldset.P[time, particle.depth, particle.lat, particle.lon]
    
def SamplePoff(particle, fieldset, time):
    """ offset sampling by dt
    """
    particle.p = fieldset.P[time+particle.dt, particle.depth, particle.lat, particle.lon]

### generate input fields


\begin{align}
x &= [0,1], \\
y &= [0,1], \\
u(x,y,t) &= 0.5, \\
v(x,y,t) &= 0, \\
p(x,y,t) &= 10t + x, \\
\end{align}

We'll follow a parcel that is initially at $x=0$ and $y=0$ with $p(x=0,y=0,t=0)=0$.

After a unit time step, the parcel is at $x=0.5$ and $y=0$ with $p(x=0.5,y=0,t=1)=10.5$


In [3]:
dims = [5, 4, 2]
dx, dy = 1./dims[0], 1./dims[1]
dimensions = {'lon': np.linspace(0., 1., dims[0], dtype=np.float32),
              'lat': np.linspace(0., 1., dims[1], dtype=np.float32),
              'time': np.arange(dims[2], dtype=np.float32),
             }

p = (10*np.arange(dims[2])[None, None, :]
     + dimensions['lon'][:, None, None]
     + np.zeros(dims[1], dtype=np.float32)[None, :, None]
    )

data = {'U': np.ones(dims, dtype=np.float32)/2,
        'V': np.zeros(dims, dtype=np.float32),
        'P': p   
       }
fieldset = FieldSet.from_data(data, dimensions, mesh='flat', transpose=True)

xv, yv = np.meshgrid(np.arange(0, 1, 0.5), np.arange(0, 1, 0.5))

---
### 1: Base configuration

In [4]:
pset = ParticleSet(fieldset, pclass=SampleParticle, lon=xv.flatten(), lat=yv.flatten())
pfile = ParticleFile("tmp1.nc", pset, outputdt=1)
pset.execute(SampleP, endtime=1, dt=1, output_file=pfile)
pfile.close()
pset

INFO: Compiled SampleParticleSampleP ==> /var/folders/pb/vp119fcj1pn3dc3x2dz4dkt80000gn/T/parcels-501/0120dab411f0fda7d0c2708c638cd1f6_0.so


P[0](lon=0.000000, lat=0.000000, depth=0.000000, p=0.000000, time=1.000000)
P[1](lon=0.500000, lat=0.000000, depth=0.000000, p=0.500000, time=1.000000)
P[2](lon=0.000000, lat=0.500000, depth=0.000000, p=0.000000, time=1.000000)
P[3](lon=0.500000, lat=0.500000, depth=0.000000, p=0.500000, time=1.000000)

In [5]:
def print_nc(n):
    ds = xr.open_dataset(f"tmp{n}.nc").isel(obs=1)
    print("lon={}".format(ds["lon"].values[0]))
    print("p={}".format(ds["p"].values[0]))
    print("time={}".format(ds["time"].values[0]/np.timedelta64(1,'s')))

print_nc(1)

lon=0.0
p=0.0
time=1.0


Information stamped at $t$ is:

- Position: $x(t-dt)$
- Variable $p$ evaluated at $x(t-dt)$ and $t-dt$

Not so bad of a situation as only the timestamp is incorrect.

---
### 2: manually sum kernels with RK4+SampleP

In [6]:
pset = ParticleSet(fieldset, pclass=SampleParticle, lon=xv.flatten(), lat=yv.flatten())
kernels = AdvectionRK4 + pset.Kernel(SampleP)
pfile = ParticleFile("tmp2.nc", pset, outputdt=1)
pset.execute(kernels, endtime=1, dt=1, output_file=pfile)
pfile.close()
pset

INFO: Compiled SampleParticleAdvectionRK4SampleP ==> /var/folders/pb/vp119fcj1pn3dc3x2dz4dkt80000gn/T/parcels-501/a14b6229de31167b0795e58411e31226_0.so


P[4](lon=0.500000, lat=0.000000, depth=0.000000, p=0.500000, time=1.000000)
P[5](lon=1.000000, lat=0.000000, depth=0.000000, p=1.000000, time=1.000000)
P[6](lon=0.500000, lat=0.500000, depth=0.000000, p=0.500000, time=1.000000)
P[7](lon=1.000000, lat=0.500000, depth=0.000000, p=1.000000, time=1.000000)

In [7]:
print_nc(2)

lon=0.5
p=0.5
time=1.0


Information stamped at $t$ is:

- Position : $x(t)$
- Variable $p$ evaluated at $x(t)$ and $t-dt$

**!Danger!: the interpolation does not use consistent position and time**

---

### 3: manually sum kernels with SampleP+RK4

(personal note: used in production run)

In [8]:
pset = ParticleSet(fieldset, pclass=SampleParticle, lon=xv.flatten(), lat=yv.flatten())
kernels = pset.Kernel(SampleP) + AdvectionRK4
pfile = ParticleFile("tmp3.nc", pset, outputdt=1)
pset.execute(kernels, endtime=1, dt=1, output_file=pfile)
pfile.close()
pset

INFO: Compiled SampleParticleSamplePAdvectionRK4 ==> /var/folders/pb/vp119fcj1pn3dc3x2dz4dkt80000gn/T/parcels-501/89eacaae3f5adca1917d7934eeff2fb2_0.so


P[8](lon=0.500000, lat=0.000000, depth=0.000000, p=0.000000, time=1.000000)
P[9](lon=1.000000, lat=0.000000, depth=0.000000, p=0.500000, time=1.000000)
P[10](lon=0.500000, lat=0.500000, depth=0.000000, p=0.000000, time=1.000000)
P[11](lon=1.000000, lat=0.500000, depth=0.000000, p=0.500000, time=1.000000)

In [9]:
print_nc(3)

lon=0.5
p=0.0
time=1.0


Information stamped at $t$ is:

- Position: $x(t)$
- Variable $p$ evaluated at $x(t-dt)$ and $t-dt$

Not so bad of a situation but need to keep in mind position and $p$ refer to differnt timestamps

---

### 4: manually sum kernels with RK4+SamplePoff

In [10]:
pset = ParticleSet(fieldset, pclass=SampleParticle, lon=xv.flatten(), lat=yv.flatten())
kernels = AdvectionRK4 + pset.Kernel(SamplePoff)
pfile = ParticleFile("tmp4.nc", pset, outputdt=1)
pset.execute(kernels, endtime=1, dt=1, output_file=pfile)
pfile.close()
pset

INFO: Compiled SampleParticleAdvectionRK4SamplePoff ==> /var/folders/pb/vp119fcj1pn3dc3x2dz4dkt80000gn/T/parcels-501/335ced1adbde87f6ebe3b6a81a495b85_0.so


P[12](lon=0.500000, lat=0.000000, depth=0.000000, p=10.500000, time=1.000000)
P[13](lon=1.000000, lat=0.000000, depth=0.000000, p=11.000000, time=1.000000)
P[14](lon=0.500000, lat=0.500000, depth=0.000000, p=10.500000, time=1.000000)
P[15](lon=1.000000, lat=0.500000, depth=0.000000, p=11.000000, time=1.000000)

In [11]:
print_nc(4)

lon=0.5
p=10.5
time=1.0


Information stamped at $t$ is:

- Position: $x(t)$
- Variable $p$ evaluated at $x(t)$ and $t$

**This appear to be the correct solution**

---

### 5: manually sum kernels with SamplePoff+RK4

In [12]:
pset = ParticleSet(fieldset, pclass=SampleParticle, lon=xv.flatten(), lat=yv.flatten())
kernels = pset.Kernel(SamplePoff) + AdvectionRK4
pfile = ParticleFile("tmp5.nc", pset, outputdt=1)
pset.execute(kernels, endtime=1, dt=1, output_file=pfile)
pfile.close()
pset

INFO: Compiled SampleParticleSamplePoffAdvectionRK4 ==> /var/folders/pb/vp119fcj1pn3dc3x2dz4dkt80000gn/T/parcels-501/2782524329af42865d9a1753a606d3fa_0.so


P[16](lon=0.500000, lat=0.000000, depth=0.000000, p=10.000000, time=1.000000)
P[17](lon=1.000000, lat=0.000000, depth=0.000000, p=10.500000, time=1.000000)
P[18](lon=0.500000, lat=0.500000, depth=0.000000, p=10.000000, time=1.000000)
P[19](lon=1.000000, lat=0.500000, depth=0.000000, p=10.500000, time=1.000000)

In [13]:
print_nc(5)

lon=0.5
p=10.0
time=1.0


Information stamped at $t$ is:

- Position: $x(t)$
- Variable $p$ evaluated at $x(t-dt)$ and $t$

**!Danger!: the interpolation does not use consistent position and time**